In [ ]:
# =========================
# Baseline multi-seed runner
# =========================
import sys, os, importlib, warnings, time, random
import numpy as np
import torch
import pandas as pd
import pandas as pd
import numpy as np
import plotly.graph_objects as go


proj_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if proj_root not in sys.path:
    sys.path.append(proj_root)

from utils.arguments import get_args
from utils.print_args import print_args
from utils.tools import get_setting
from exp.trainer_forecasting import Trainer_Forecasting

print("[Path] proj_root =", proj_root)
print("[Torch] version:", torch.__version__)
print("[Torch] cuda.is_available():", torch.cuda.is_available())

# ---------- helpers ----------
def compute_hit_rates(full_pred: torch.Tensor, full_true: torch.Tensor, tolerances=(0.10, 0.05, 0.03)):
    if full_true.is_cuda: full_true = full_true.cpu()
    if full_pred.is_cuda: full_pred = full_pred.cpu()
    num_total = full_true.numel()
    if num_total == 0:
        return {"num_total": 0}
    abs_err = torch.abs(full_pred - full_true)
    out = {"num_total": int(num_total)}
    for tol in tolerances:
        hits = torch.sum(abs_err <= tol).item()
        out[f"hits@±{tol}"] = int(hits)
        out[f"hit_rate@±{tol}"] = 100.0 * hits / num_total
    return out

def set_global_seed(seed: int):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def _build_trainer(args, df_raw):
    exp = Trainer_Forecasting(args, df_raw)
    device = torch.device('cuda' if getattr(args, 'use_gpu', False) and torch.cuda.is_available() else 'cpu')
    if hasattr(exp, 'model') and exp.model is not None:
        exp.model = exp.model.to(device)
    exp.device = device
    return exp

def run_one_baseline_once(df_raw, model_id: str, args_common, tolerances=(0.10, 0.05, 0.03)):
    import copy
    args = copy.deepcopy(args_common)
    args.model = model_id

    feature_dim = df_raw.shape[-1] - 1
    args.enc_in = feature_dim
    args.dec_in = feature_dim
    args.c_out = 1 if args.features == 'MS' else feature_dim

    setting = get_setting(args)
    exp = _build_trainer(args, df_raw)

    model, history = exp.train(setting)
    preds, trues, full_pred, full_true, metrics, base_dir, svg_path = exp.test(setting)

    hit_stats = compute_hit_rates(full_pred, full_true, tolerances=tolerances)

    row = {
        "model": model_id,
        "MAE": float(metrics.get("mae", metrics.get("MAE", float("nan")))),
        "RMSE": float(metrics.get("rmse", metrics.get("RMSE", float("nan")))),
        "MAPE": float(metrics.get("mape", metrics.get("MAPE", float("nan")))),
        "num_total": hit_stats.get("num_total", 0),
    }
    for k, v in hit_stats.items():
        if k != "num_total":
            row[k] = v
    return row

def run_baselines_multi_seed(
    df_raw,
    baselines,
    seeds=(0,1,2,3,4),
    tolerances=(0.10, 0.05, 0.03),
    save_prefix="baseline_multiseed",
):
    parser = get_args()
    args = parser.parse_args([
        '--task_name', 'forecast',
        '--seq_len', '168',
        '--label_len', '0',
        '--pred_len', '24',          # 统一 horizon，便于公平比较
        '--use_gpu', 'True',
        '--num_workers', '0',
        '--features', 'MS',
        '--inverse',

        '--d_model','512',
        '--d_ff', '4',
        '--e_layers', '3',
        '--d_layers', '3',

        # 给 TimeMixer / ModernTCN 等需要的超参（即便不用也不会出错）
        '--down_sampling_layers', '3',
        '--down_sampling_window', '2',
        '--down_sampling_method', 'avg',

        '--ffn_ratio', '1',
        '--patch_size', '16',
        '--patch_stride', '8',
        '--num_blocks', '1',
        '--large_size', '51',
        '--small_size', '5',
        '--dims', '1',
        '--use_multi_scale', 'False',
        '--dropout', '0.05',
        '--head_dropout', '0.0',

        '--train_epochs', '300',
        '--batch_size', '64',
        '--learning_rate', '1e-4',
        '--optimizer', 'AdamW',
        '--lradj', 'CosineAnnealingLR',
        '--loss', 'L1',

        '--train_ratio', '0.8',
        '--val_ratio', '0.1',
        '--test_ratio', '0.1',
        '--patience', '10',
        '--weight_decay', '0.001',
        '--lr_decay_min_lr', '1e-5',
        '--lr_decay_factor', '0.9',
    ])
    print_args(args)

    all_summary_rows = []
    os.makedirs(".", exist_ok=True)

    for model_name in baselines:
        print("\n" + "#"*100)
        print(f"▶ Baseline model: {model_name}")
        print("#"*100)

        per_seed_rows = []
        for seed in seeds:
            print(f"[Seed={seed}] ------------------------------")
            set_global_seed(seed)
            row = run_one_baseline_once(
                df_raw=df_raw,
                model_id=model_name,
                args_common=args,
                tolerances=tolerances
            )
            per_seed_rows.append(row)

            # 每个 seed 单独保存
            pd.DataFrame([row]).to_csv(
                f"{save_prefix}_{model_name}_seed{seed}.csv",
                index=False, encoding="utf-8-sig"
            )

            # 简要打印
            print(f"  MAE={row['MAE']:.6f}, RMSE={row['RMSE']:.6f}, MAPE={row['MAPE']:.6f}")
            for tol in tolerances:
                print(f"  Hit@±{tol}: {row.get(f'hit_rate@±{tol}', float('nan')):.2f}%")

        # 聚合 mean ± std
        dfm = pd.DataFrame(per_seed_rows)
        summary = {"model": model_name}
        for col in ['MAE', 'RMSE', 'MAPE',
                    'hit_rate@±0.03', 'hit_rate@±0.05', 'hit_rate@±0.1']:
            if col in dfm.columns:
                summary[col] = f"{dfm[col].mean():.6f} ± {dfm[col].std():.6f}"
        all_summary_rows.append(summary)

    df_summary = pd.DataFrame(all_summary_rows)
    out_path = f"{save_prefix}_summary.csv"
    df_summary.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"\n✅ Baseline multi-seed summary saved to: {os.path.abspath(out_path)}")
    return df_summary


In [ ]:
# 读取处理后数据（最后一列为 SiContent）
df_raw = pd.read_csv("new_ETTh1.csv", encoding="utf-8-sig")

# 选择要对比的 baseline（名字与 Trainer_Basic.model_dict 完全一致）
baselines = [
    'GRU', 'oneD_CNN', 'LSTM', 
    # 'XGBoost',  # 经典/浅层
    'DLinear', 'ModernTCN_forecasting', 
    # 'TimeMixer', 
    'TemporalFusionTransformer',
    'Informer', 'TimeXer'                   # SOTA系
]

results = run_baselines_multi_seed(
    df_raw=df_raw,
    baselines=baselines,
    seeds=(0,1,2,3,4),
    tolerances=(0.10, 0.05, 0.03),
    save_prefix="baselines_multiseed"
)

results
